# MNIST Syft Data Owner

In [ ]:
%%capture
# this only runs in colab and clones the code sets it up and fixes a few issues
import sys
if "google.colab" in sys.modules:
    branch = "demo"    # change to the branch you want
    ! git clone --single-branch --branch $branch https://github.com/OpenMined/PySyft.git
    ! cd PySyft && ./scripts/colab.sh.     # fixes some colab python issues
    sys.path.append("/content/PySyft/src") # prevents needing restart

In [ ]:
import syft as sy
sy.LOG_FILE = "syft_do.log"
_ = sy.logger.add(sy.LOG_FILE, enqueue=True, colorize=False, diagnose=True, backtrace=True, level="TRACE")

# PART 1: Launch a Duet Server and Connect

As a data owner, you want to allow someone else to perform data science on data that you own and likely want to protect.

In order to do this, we must load our data into a locally running server (within this notebook).
We call this server a "Duet".

To begin, you must launch Duet and help your Duet "partner" (a data scientist) connect to this server.

You do this by running the code below and following the instructions it gives!

In [ ]:
duet = sy.launch_duet(loopback=True, db_path="./syft.sqlite")
# duet = sy.launch_duet(loopback=True)
# duet = sy.duet()

In [ ]:
duet.store.pandas

In [ ]:
# accept all incoming requests named age_data, print and log them
duet.requests.add_handler(
    name="age_data",
    action="accept",
    element_quota=100,
)

In [ ]:
duet.requests.add_handler(
    name="cuda_is_available",
    action="accept"
)

duet.requests.add_handler(
    name="loss",
    action="deny",
    timeout_secs=-1,
    print_local=True,
    log_local=True
)

duet.requests.add_handler(
    name="train_size",
    action="accept"
)

duet.requests.add_handler(
    name="inference",
    action="accept"
)

duet.requests.add_handler(
    name="model_download",
    action="accept"
)

In [ ]:
duet.requests.remove_handler(
    name="model_download",
    action="accept"
)

In [ ]:
duet.requests.handlers

In [ ]:
for handler in duet.requests.handlers:
    new_dict = {}
    del handler["created_time"]
    for k, v in handler.items():
        new_dict[str(k)] = v
    duet.requests.remove_handler(**new_dict)

If you are in Jupyter Notebook (not Colab) the ☝🏾DUET LIVE STATUS above will animate 

# PART 2: Launch a Duet Server and Connect

In [ ]:
import torch as th
import torchvision as tv

In [ ]:
# age_data = th.rand((4608, 128))   # 589824 params # works Wall time: 24.7 s
age_data = th.rand((10, 1))   # params # works Wall time: 24.7 s
print(age_data.nelement())
age_data = age_data.tag(f"size_{age_data.nelement()}")
age_data_pointer = age_data.send(duet, searchable=True)

In [ ]:
age_data = th.rand((1, 1)) # works Wall time: 46.6 s
age_data = age_data.tag("age_data")
age_data_pointer = age_data.send(duet, searchable=True)

In [ ]:
duet.store.pandas